In [1]:
# start

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5,application_1727235573085_0007,pyspark,idle,Link,Link,user_n-renukumar_rajendran_draup_com,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc.addPyFile("s3://jobs-emr-etl-config/jobs_run/prod/cc/ML_ETL_Config.py")
from ML_ETL_Config import *
utils_module = MlEtlUtils(spark, sc)
to_import = [name for name in dir(utils_module) if not (name.startswith('_') | (name=="spark") | (name=="sc"))]
globals().update({name: getattr(utils_module, name) for name in to_import})
from pyspark.sql.functions import *
from pyspark.sql import Window

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

404
404
/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gateway.draup.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gateway.draup.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(

In [4]:
pce_df = spark.read.format("delta").load("s3://etl-datahouse/prod/profiles/pce/pce_calc/pce_snapshot/delta").select('mvp_company_name','currentPositions','pastPositions','publicLink','functional_workload','business_function','core_Skills','msa','years_of_experience','publications','patents','certifications').selectExpr('*','currentPositions[0].companyName as curr_company','currentPositions[0].company_industry as current_industry','currentPositions[0].job_title_lemmatized as current_job_title','currentPositions.job_role[0] as current_job_role','currentPositions[0].msa as current_msa','currentPositions[0].i18nStartDate as current_StartDate','currentPositions[0].i18nEndDate as current_EndDate','currentPositions[0].startDateYear as startDateYear','currentPositions[0].endDateYear as endDateYear','pastPositions[0].companyName as past_company','pastPositions[0].company_industry as past_industry','pastPositions[0].experience_years as past_company_experience','pastPositions[0].msa as past_msa','patents[0].title as patents_title','publications[0].title as publication_title','certifications[0].name as certifications_name').drop('currentPositions','pastPositions','past_data','deleted','patents','publications','certifications')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#pce_df.write.mode("overwrite").parquet('s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/pce_keyword_data_task_25-09-2024')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
pce_data = spark.read.parquet('s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/pce_keyword_data_task_25-09-2024')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
[PATH_NOT_FOUND] Path does not exist: s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/pce_keyword_data_task_18-09-2024.
Traceback (most recent call last):
  File "/mnt/yarn/usercache/user_n-renukumar_rajendran_draup_com/appcache/application_1727235573085_0007/container_1727235573085_0007_01_000001/pyspark.zip/pyspark/sql/readwriter.py", line 544, in parquet
    return self._df(self._jreader.parquet(_to_seq(self._spark._sc, paths)))
  File "/mnt/yarn/usercache/user_n-renukumar_rajendran_draup_com/appcache/application_1727235573085_0007/container_1727235573085_0007_01_000001/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/mnt/yarn/usercache/user_n-renukumar_rajendran_draup_com/appcache/application_1727235573085_0007/container_1727235573085_0007_01_000001/pyspark.zip/pyspark/errors/exceptions/captured.py", line 185, in deco
    raise converted from None
pyspark.errors.exceptions.captured

In [23]:
keyword = ['Innovation', 'Senior Innovation', 'R&D', 'Research and Development', 'Technology Innovation', 'Product Innovation', 'Innovation & Design', 'Flexible Packaging Solutions', 'Innovation and Market Development', 'Senior Product Innovation', 'Flexible Packaging Innovation', 'Research Innovation', 'Marketing Innovation', 'R&D Innovation', 'Research and Development Innovation', 'R&D Global Packaging Innovation', 'Innovation Projects', 'Technical']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
include_labels = ['Manager', 'Lead', 'Leader', 'Team Lead']
exclude_labels = ['owner', 'Trainee']


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
df_filtered = pce_data.filter(
    F.expr("array_contains(array({}), current_job_title)".format(
        ','.join(f"'{kw}'" for kw in keyword)
    ))
)

# Step 2: Extract those that contain any of the include labels
df_extracted = df_filtered.filter(
    F.expr("array_contains(array({}), current_job_title)".format(
        ','.join(f"'{label}'" for label in include_labels)
    ))
)

# Exclude those that contain any of the exclude labels
df_excluded = df_filtered.filter(
    ~F.expr("array_contains(array({}), current_job_title)".format(
        ','.join(f"'{label}'" for label in exclude_labels)
    ))
)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
Keywords_profile = ['polymers', 'packaging', 'food packaging', 'plastic packaging', 'plastics', 'Package Technology', 'Flexibles']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
df_with_combined = df_excluded.withColumn(
    "combined_data",

    concat_ws(", ", "curr_company","core_Skills","current_job_title","current_msa","patents_title","publication_title","certifications_name")
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
filtered_df = df_with_combined.filter(
    df_with_combined.combined_data.contains("Polymers") |
    df_with_combined.combined_data.contains("Packaging") |
    df_with_combined.combined_data.contains("Food Packaging") |
    df_with_combined.combined_data.contains("Plastics Packaging") |
    df_with_combined.combined_data.contains("Plastics") |
    df_with_combined.combined_data.contains("Package Technology") |
    df_with_combined.combined_data.contains("Flexibles")
)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
filtered_df[["combined_data"]].show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|combined_data                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
+-------------------------------------------

In [47]:
filtered_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

58

### Keyword Match(Inbount Migration)

In [33]:
pce_data1 =pce_data.filter(col('msa') == ('National Capital Region, Philippines'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
keyword_df = pce_data1.withColumn(
    'keyword',
    concat_ws(", ", "curr_company","Core_Skills","current_job_title","current_msa","patents_title","publication_title","certifications_name"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
filter_df = keyword_df.filter(
    (col('keyword').rlike("Customer Care|Client Support|Customer Success|Customer&Python")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
filter_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

9284

#### Keyword(outbount Migration)

In [24]:
pce_data2 =pce_data.filter(col('past_msa') == ('National Capital Region, Philippines'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
keyword_df = pce_data2.withColumn(
    'keyword', 
    concat_ws(", ", "curr_company","Core_Skills","current_job_title","current_msa","patents_title","publication_title","certifications_name"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
outbount_data = keyword_df.filter(
    (col('keyword').rlike("Customer Care|Client Support|Customer Success|Customer&Python")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
outbount_data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2290